## Things do not understand

1. sequential API vs functional API
2. Siames Neural Network
3. convolutional nural newtwork
4. Memory growth of GPU 
5. virtual environment

# Setup

## Install Dependencies

In [1]:
!pip install tensorflow==2.4.1 tensorflow-gpu==2.4.1 opencv-python matplotlib

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 394.3 MB 12 kB/s 
     |████████████████████████████████| 394.3 MB 15 kB/s 
     |████████████████████████████████| 3.8 MB 47.4 MB/s 
     |████████████████████████████████| 132 kB 62.8 MB/s 
     |████████████████████████████████| 2.9 MB 75.3 MB/s 
     |████████████████████████████████| 14.8 MB 43.9 MB/s 
     |████████████████████████████████| 462 kB 70.2 MB/s 
  Created wheel for wrapt: filename=wrapt-1.12.1-cp37-cp37m-linux_x86_64.whl size=68715 sha256=beee229557bbad01c66996c3557db2dc5bd0c73a4b18d444829206b12d4e8c7f
  Stored in directory: /root/.cache/pip/wheels/62/76/4c/aa25851149f3f6d9785f6c869387ad82b3fd37582fa8147ac6
Successfully built wrapt
  Attempting uninstall: typing-extensions
    Found existing installation: typing-extensions 4.1.1
    Uninstalling typing-extensions-4.1.1:
      Successfully uninstalled typing-extensions-4.1.1
  Atte

## import dependencies

Standerd dependancies

In [3]:
# opencv will give us image process capabilities, and we needs it for testing
import cv2
# it abstact the OS from our python code! makes it a whole heap, and easier to work with different directories and different file paths
import os
# ganerate random data
import random
# great when working with arrays
import numpy as np
# visualization
from matplotlib import pyplot as plt

### Transorflow Ddpendencies

there is a sequintial API, and functional API. fancional API, usefull for hardcore deep learning models


In [5]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Layer, Conv2D, Dense, MaxPooling2D, Input, Flatten
import tensorflow as tf

Model: Model(inputs=[inputimage, verificationimage], outputs=[0,1])

Layer: high level class, allows to define a cusom layer, class L1Dist(Layer)

Conv2v: allow convolution, convolutional nural newtwork

Dense: give us fully connected layer

MaxPooling: shrinks the information 

Input: Input(shape=)

flatten: flatten the data into a single dimension layer, allows to pass the convolutional neural network data to dense layer

## Set GPU Growth

limite how much vram tensorflow can use, don't let TF to go crazy

In [6]:
# avoind OOM errors by setting GPU Memory Consumption Growth
# giting all available GPUs on our machien
gpus = tf. config.experimental.list_physical_devices('GPU')
for gpu in gpus:
  tf.config.experimental.set_memory_growth(gpu,True)

In [9]:
len(gpus)

1

## Create folder Structures

Ancher: Unknown Input

Positive: What should match

Nigative: What should not match

In [10]:
# Setup paths
POS_PATH = os.path.join('data', 'positive')
NEG_PATH = os.path.join('data', 'negaitive')
ANC_PATH = os.path.join('data', 'anchoer')

In [11]:
# Mke the directories
os.makedirs(POS_PATH)
os.makedirs(NEG_PATH)
os.makedirs(ANC_PATH)